In [2]:
%pylab qt
import csv
#to change working directory
#import os
#os.chdir(r'D:\!Data\2015_09_22')

filename = 'tek00023.csv'

#only for 2 coloms
x = []
y = []
with open(filename, 'r') as csvfile:
    a = csv.reader(csvfile)
    for line in a:
        x.append(float(line[0]))
        y.append(float(line[1]))        
plot(x,y)

Populating the interactive namespace from numpy and matplotlib


In [98]:
# x, y - data to prosess, should be initialize befor

# I assume that  command below was executed, qt - to show plots in separate window, use 'inline' instead to plot here 
#%pylab qt
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter
from inspect import getargspec

# Gaussian function
def gaussian(x,N,x0,sigma, background):
    """Returns value of a 1D-gaussian with the given parameters ,N,x0,sigma, background"""
    #from numpy import sqrt,pi,exp
    return N / (sigma * sqrt(pi)) * exp(-(x - x0)**2/(sigma**2)) + background

# lorentz function
def lorenz(x, a, x0, g, bg):
    return a * g/2 / ((x-x0)**2 + g**2/4) + bg

# If filtering is needed
y = gaussian_filter(y,20)

# threshold value to find piks and then to estimate width
thr = 0.04 #max(y)/2

# construct list of indexis y[index] > threshold
n1 = [x for (x,val) in enumerate(y) if val > thr] 
#print(n1)

# construct coordinates of beggining and ending of the pick based on threshold value
coords = []
for num,elem in enumerate(n1):
    if num == 0:   # iteration start
        st = 0
    elif elem - n1[num-1] > 400:    # if jump to next pick, number specifies minimum distance between peaks
        coords.append((n1[st],n1[num-1]))
        st = num
    elif  num == len(n1)-1:       # if came to list end
        coords.append((n1[st],n1[num]))
# print(coords)        

# fitting of individual peak in the neghboring of n * width and plotting them
n = 8     #what range to use to find fit
plot(x,y)
popts = []
# fit_func = lorenz
fit_func = gaussian
print('Fit parameters:',getargspec(fit_func)[0][1:])
for num,(lw,up) in enumerate(coords):
    width = abs(up - lw)
    sls = slice(max(0,lw - n*width), min(up + n*width,len(x)))
    popt, pcov = curve_fit(fit_func, x[sls], y[sls],p0=(0.001, (x[up]+x[lw])/2, (x[up]-x[lw]),0.023))
    plot(x[sls],fit_func(x[sls],*popt))
    popts.append(popt)
    annotate(str(num),xy=(popt[1],0))
    print(num, '---->', popt)

Fit parameters: ['N', 'x0', 'sigma', 'background']
0 ----> [  8.62960080e-05  -4.76021548e-01   1.68389978e-03   2.32134825e-02]
1 ----> [  9.42841711e-05  -4.08522049e-01   1.79990482e-03   2.29792936e-02]
2 ----> [ 0.00065744 -0.35710507  0.001479    0.02401031]
3 ----> [ 0.00082832 -0.30239619  0.00161387  0.02471149]
4 ----> [  9.68281261e-05  -2.49887271e-01   2.06219591e-03   2.28649213e-02]
5 ----> [  8.96647832e-05  -1.83177372e-01   1.53308429e-03   2.31170372e-02]
6 ----> [  8.55438481e-05  -1.18061215e-01   1.59277534e-03   2.29872333e-02]
7 ----> [  9.89347898e-05  -5.05673506e-02   1.87724784e-03   2.29982207e-02]
8 ----> [ 0.00080095  0.00104816  0.00173151  0.02427776]
9 ----> [ 0.00057398  0.05546427  0.00148017  0.02427074]
10 ----> [  1.01144760e-04   1.07934466e-01   1.92271950e-03   2.27311149e-02]
11 ----> [  8.05521856e-05   1.74495898e-01   1.60587359e-03   2.33780983e-02]
12 ----> [  8.94419058e-05   2.39822736e-01   1.75161373e-03   2.32739407e-02]
13 ----> [  

In [94]:
# to calculate relation between peak width and distance 
n0 = 12
n1 = 14
abs(popts[n0][1] - popts[n1][1]) / popts[n0][2]

67.987105926348761